In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import string

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

# Hypothesis Testing

**Hypothesis**: University towns have their mean housing prices less effected by recessions. 

Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. We only look at GDP data from the first quarter of 2000 onward.

In [2]:
#  dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
#Cleaning GDP Data to only consider year 2000 onwards
GDP = pd.read_excel('gdplev.xls')
GDP = GDP.drop(GDP.columns[0:4],axis = 1)
GDP = GDP.drop(GDP.columns[1],axis =1)
GDP = GDP.drop(GDP.columns[2],axis =1)
GDP = GDP[219:]
GDP.columns = ['quarter','USD Billion']
GDP['change'] = GDP['USD Billion'].diff(1)
GDP = GDP.reset_index()
GDP = GDP.drop(GDP.columns[0], axis = 1)

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns = pd.read_table('university_towns.txt', header = None)
    towns = pd.DataFrame(columns=["State", "RegionName"])
    i = 0
    for row in university_towns[0]:
        if (("[" in row) & ("(" not in row)):
            state = (row.split('[')[0]).strip()
        else:
            region = (row.split('(')[0]).strip()
            towns.loc[i] = [state,region]
            i += 1
    
    return towns
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    for i in range(len(GDP['change'][:-2])):
        if ((GDP['change'].iloc[i] < 0) & (GDP['change'].iloc[i+1] < 0)):
            return (GDP['quarter'].iloc[i])
        
get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    rs = GDP['quarter'][GDP['quarter'] == get_recession_start()].index.tolist()[0]
    
    for i in range(rs, len(GDP['change'][:-2])):
        if ((GDP['change'].iloc[i] > 0) & (GDP['change'].iloc[i+1] > 0)):
            return (GDP['quarter'].iloc[i+1])
        
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    rs = GDP['quarter'][GDP['quarter'] == get_recession_start()].index.tolist()[0]
    re = GDP['quarter'][GDP['quarter'] == get_recession_end()].index.tolist()[0]
    
    lowest_GDP = GDP['USD Billion'].iloc[rs]
    
    for i in range(rs+1,re-1): #start low (rs) already taken. poitive growth already started in (re-1) 
        gdp = GDP['USD Billion'].iloc[i]
        if gdp < lowest_GDP:
            lowest_GDP = gdp
    
    return GDP['quarter'][GDP['USD Billion'] == lowest_GDP].tolist()[0]
get_recession_bottom()

'2009q2'

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data['State'] = housing_data['State'].replace(states)
    housing_data = housing_data.drop(housing_data.columns[0],axis = 1)
    housing_data = housing_data.drop(housing_data.columns[2:50],axis = 1)
    housing_data = housing_data.set_index(['State','RegionName'])
    housing_data.columns = pd.to_datetime(housing_data.columns)

    housing_data = housing_data.resample('Q',axis=1).mean()
    housing_data = housing_data.rename(columns=lambda x: str(x.to_period('Q')).lower())
    
    return housing_data
convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [10]:
rs = get_recession_start()
rb = get_recession_bottom()
rs_yr = int(rs[0:4])
rs_q = int(rs[-1])

if rs_q == 1:
    prev_q = 4
    rs_yr = rs_yr -1
else:
    prev_q = rs_q - 1
    
prev_period = str(rs_yr) + 'q' + str(prev_q)

data = convert_housing_data_to_quarters()

rb_data = data[rb]
prev_data = data[prev_period]

data['ratio'] = prev_data/rb_data

univ_towns = get_list_of_university_towns().set_index(['State','RegionName'])
univ_towns['type'] = 'uni'

univ = pd.merge(univ_towns,data['ratio'].to_frame(),how = 'inner',left_index = True, right_index = True)
common = pd.merge(univ_towns,data['ratio'].to_frame(),how = 'right',left_index = True, right_index = True)

common['type'] = common['type'].fillna('non-uni')
common.head(3)

non_univ = common[common['type'] == 'non-uni']
non_univ = non_univ.drop(non_univ.columns[0],axis = 1)


In [11]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    stat,p = ttest_ind(univ['ratio'],non_univ['ratio'],nan_policy="omit")

    if p < 0.01:
        different = True
    else:
        different = False
        
    if univ['ratio'].mean() < non_univ['ratio'].mean():
        better = "university town"
    else:
        better = "non-university town"
    
    return (different, p , better)
run_ttest()

(True, 0.002724063704753125, 'university town')